In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('data_final.csv')

In [4]:
data.shape

(1220857, 23)

In [5]:
data.columns

Index(['loan_amnt', 'term', 'int_rate', 'installment', 'sub_grade',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'loan_status', 'purpose', 'addr_state', 'inq_last_6mths', 'open_acc',
       'pub_rec', 'total_acc', 'initial_list_status', 'application_type',
       'acc_now_delinq', 'mort_acc', 'pub_rec_bankruptcies', 'dti_nr',
       'fico_score'],
      dtype='object')

In [6]:
data.home_ownership.value_counts()

MORTGAGE    609003
RENT        485696
OWN         125806
OTHER          352
Name: home_ownership, dtype: int64

In [7]:
data.rename(columns = {'loan_status':'charged-off'}, inplace = True) 

In [8]:
data.columns

Index(['loan_amnt', 'term', 'int_rate', 'installment', 'sub_grade',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'charged-off', 'purpose', 'addr_state', 'inq_last_6mths', 'open_acc',
       'pub_rec', 'total_acc', 'initial_list_status', 'application_type',
       'acc_now_delinq', 'mort_acc', 'pub_rec_bankruptcies', 'dti_nr',
       'fico_score'],
      dtype='object')

In [9]:
data['charged-off'].value_counts()

Fully Paid     980032
Charged Off    240825
Name: charged-off, dtype: int64

In [10]:
data.loc[(data['charged-off'] == 'Fully Paid'),'charged-off']= 0

In [11]:
data.loc[(data['charged-off'] == 'Charged Off'),'charged-off']= 1

In [12]:
data['charged-off'].value_counts()

0    980032
1    240825
Name: charged-off, dtype: int64

In [13]:
data.groupby('charged-off').mean()

,loan_amnt,term,int_rate,installment,emp_length,annual_inc,inq_last_6mths,open_acc,pub_rec,total_acc,acc_now_delinq,mort_acc,pub_rec_bankruptcies,dti_nr,fico_score
charged-off,,,,,,,,,,,,,,,
0,14404.165706,40.972270,12.654367,439.472596,6.113580,79569.269730,0.617621,11.672046,0.207621,25.234883,0.005152,1.737273,0.128464,17.850978,699.471664
1,15958.883733,45.702249,15.802110,476.334244,5.966237,72680.877105,0.776556,12.110549,0.244671,24.953975,0.005855,1.361275,0.150267,20.137160,689.246986


In [13]:
data = pd.get_dummies(data, columns=['sub_grade', 'home_ownership', 'verification_status', 
                                                 'purpose', 'addr_state', 'initial_list_status', 
                                                 'application_type'], drop_first=True)

In [14]:
data.shape

(1220857, 23)

## Training

In [15]:
data1 = data.sample(frac=0.95, random_state=786)
data1_unseen = data.drop(data1.index).reset_index(drop=True)
data1.reset_index(drop=True, inplace=True)

print('Data for Modeling: ' + str(data1.shape))
print('Unseen Data For Predictions: ' + str(data1_unseen.shape))

Data for Modeling: (1159814, 23)
Unseen Data For Predictions: (61043, 23)


In [53]:
pip install pycaret

  Using cached https://files.pythonhosted.org/packages/c7/41/f7fa05b6ce3cb3096a35fb5ac6dc0f2bb23e8304f068618fb2501be0a562/pycaret-1.0.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/70/cf/87b25b265d23498b2b70ce873495cf7ef91394c4baff240210e26f3bc18a/gensim-3.8.3-cp37-cp37m-macosx_10_9_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/79/c0/f7d8a0eb41ac6f302b4bc100f91b6e0f2558425ccfefaa0ec0430f77ee97/kmodes-0.10.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/7a/eb/a691d6a5f9d7e35771742ae48498d14d5661429eaa39ee9f1ca32b8e6fc6/shap-0.32.1-cp37-cp37m-macosx_10_7_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/64/e2/1610a86284029abcad0ac9bc86cb19f9787fe6448ede467188b2a5121bb4/mlxtend-0.17.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/60/f0/1d9bfcc8ee6b83472ec571406bd0dd51c0e6330ff1a51b2d29861d389e85/textblob-0.15.3-py2.py3-none-any.whl
  Using cached https://files.pythonh

  Using cached https://files.pythonhosted.org/packages/33/2a/a309969dbe9ac0738d1688e1653cf0f5cd3e511da2db986b831e986c44c2/tbb-2019.0-py2.py3-none-macosx_10_12_intel.macosx_10_12_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/77/a4/58be66c7e6520fd1e369361be3b3a81549f1f796c0b00d7109b45b33144d/boto3-1.13.2-py2.py3-none-any.whl


ERROR: phik 0.9.11 has requirement joblib>=0.14.1, but you'll have joblib 0.13.2 which is incompatible.
ERROR: umap-learn 0.4.2 has requirement numba!=0.47,>=0.46, but you'll have numba 0.45.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [16]:
from pycaret.classification import *

In [18]:
exp1 = setup(data1, target = 'charged-off')

 
Setup Succesfully Completed!


,Description,Value
0,session_id,8878
1,Target Type,Binary
2,Label Encoded,None
3,Original Data,"(1159814, 23)"
4,Missing Values,False
5,Numeric Features,14
6,Categorical Features,8
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [19]:
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,Light Gradient Boosting Machine,0.806,0.7213,0.0745,0.5605,0.1315,0.0894
1,CatBoost Classifier,0.806,0.7249,0.0883,0.5502,0.1522,0.1033
2,Gradient Boosting Classifier,0.8057,0.7167,0.0652,0.5625,0.1168,0.0791
3,Extreme Gradient Boosting,0.8054,0.7164,0.0565,0.5675,0.1027,0.0695
4,Ada Boost Classifier,0.805,0.7124,0.0673,0.5455,0.1197,0.0799
5,Ridge Classifier,0.8044,0,0.0333,0.5684,0.0629,0.042
6,Linear Discriminant Analysis,0.8034,0.7158,0.1207,0.5059,0.1949,0.1287
7,Logistic Regression,0.8033,0.6961,0.0448,0.5142,0.082,0.0522
8,Extra Trees Classifier,0.8019,0.6922,0.0866,0.4867,0.147,0.0929
9,Quadratic Discriminant Analysis,0.7993,0.5123,0.0132,0.2763,0.0246,0.0085


In [20]:
catboost = create_model('catboost')

,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,0.8053,0.7263,0.0879,0.5383,0.1511,0.1013
1,0.8059,0.7233,0.0899,0.5482,0.1545,0.1048
2,0.8068,0.7283,0.0885,0.5644,0.1530,0.1052
3,0.8051,0.7214,0.0881,0.5348,0.1512,0.1011
4,0.8064,0.7239,0.0881,0.5580,0.1521,0.1040
5,0.8067,0.7196,0.0904,0.5614,0.1557,0.1068
6,0.8056,0.7271,0.0847,0.5463,0.1467,0.0990
7,0.8066,0.7287,0.0912,0.5580,0.1568,0.1073
8,0.8063,0.7223,0.0900,0.5546,0.1548,0.1056
9,0.8052,0.7277,0.0845,0.5377,0.1461,0.0978


In [21]:
tuned_catboost = tune_model('catboost')

,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,0.8065,0.7249,0.0822,0.5643,0.1436,0.0983
1,0.8055,0.7229,0.0806,0.5466,0.1404,0.0946
2,0.8069,0.7273,0.0812,0.5735,0.1423,0.0981
3,0.8049,0.7180,0.0758,0.5377,0.1328,0.0884
4,0.8074,0.7225,0.0824,0.5815,0.1444,0.1004
5,0.8062,0.7179,0.0812,0.5587,0.1418,0.0966
6,0.8056,0.7264,0.0760,0.5522,0.1336,0.0902
7,0.8073,0.7287,0.0816,0.5799,0.1431,0.0993
8,0.8067,0.7206,0.0793,0.5704,0.1393,0.0958
9,0.8060,0.7256,0.0766,0.5576,0.1347,0.0915
